#### Metadata


- link for sublime like multiedit: https://stackoverflow.com/questions/41553806/atom-sublime-like-multiple-selections-in-jupyter

In [193]:
import time
import requests
import json
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tn
from lxml import html
import pandas as pd

In [194]:
web_link='http://www.gutenberg.org/ebooks/bookshelf/'
gutenburg_data=requests.get(web_link)
print(gutenburg_data.status_code)

200


In [195]:
soup=bs(gutenburg_data.text,features='html.parser')
#print(soup.prettify())

#### functions for making urls clickalbe and short(bitly)

- bitly_api import error solution: https://github.com/bitly/bitly-api-python/issues/39

In [196]:
def make_clickable(val):
    # target _blank to open new window
    if val!='table header: no link':
        return '<a target="_blank" href="{}">{}</a>'.format(val, val) # target blank given to open link in new tab
    else:
        return val

BITLY_ACCESS_TOKEN =input('give your bitly token:')
def shortner(url): # refer to above markdown link
    paras={
        'longurl':url,
        'access_token':BITLY_ACCESS_TOKEN
    }
    
    endpoint = 'https://api-ssl.bitly.com/v3/shorten'
    r=requests.get(endpoint,params=paras,verify=True) #gives not verifies error and 1000 links for free dahhh
    
    data=r.json()
    
    if not r.status_code==200:
        logger.error("Unexpected status_code: {} in bitly response. {}".format(data['status_code'], r.text))
    
    return make_clickable(data['data']['url']) # also making shortened links clickable

#### Creating df for home page PG main bookshelves

In [197]:
## main bookshelves
main_bookshelve_names=[]
main_bookshelve_links=[]
for link in soup.find('div',{'class':'bookshelves'}).find_all('a'):
    main_bookshelve_links.append('https://www.gutenberg.org'+link.get('href'))
    main_bookshelve_names.append(link.string)
    
main_bookshelve_df=pd.DataFrame({'Title':main_bookshelve_names,'Link':main_bookshelve_links})

In [198]:
main_bookshelve_df=main_bookshelve_df.style.format({'Link':shortner})

In [199]:
#main_bookshelve_df.to_csv('gutenburg-bookshelves-title-link(bitly).csv') # it tkaes long time as there are 22 links so sve it into csvfile

#### Creating df for home page PG sub bookshelves

In [200]:
bookshelves_pages_links=[]
bookshelves_pages_titles=[]
for link in soup.find('div',{'class':'bookshelf_pages'}).find_all('a'):
    bookshelves_pages_links.append('https://www.gutenberg.org'+link.get('href'))
    bookshelves_pages_titles.append(link.get('title'))

In [201]:
bookshelves_pages_df=pd.DataFrame({'Title':bookshelves_pages_titles,'Link':bookshelves_pages_links})

def make_clickable(val):
    # target _blank to open new window
    if val!='table header: no link':
        return '<a target="_blank" href="{}">{}</a>'.format(val, val) # target blank given to open link in new tab
    else:
        return val

bookshelves_pages_df=bookshelves_pages_df.style.format({'Link':make_clickable})

### Creating df for all PG books

In [202]:
no_of_bookshelves_pages=input('How many bookshelves pages do you want to list? ')
bookshelves_page=[]
book_name=[]
book_link=[]
book_cover_link=[]
author=[]
downloads_count=[]
for i in range(len(bookshelves_pages_links[:int(no_of_bookshelves_pages)])):
    start_time=time.time()
    print(i+1,')   Bookshelve -->',bookshelves_pages_titles[i].lower(),'<-- is being processed.......')
    page_link=bookshelves_pages_links[i]
    page_data=requests.get(page_link)
    page_soup=bs(page_data.text,features='html.parser')
    ## this loop needs to go into a function
    ## for good so that it becomes a bit more short and compact
    for book in page_soup.find_all('li',{'class':'booklink'}):
        bookshelves_page.append(bookshelves_pages_titles[i])
        book_link.append('https://www.gutenberg.org'+book.find('a').get('href'))
        if book.find('img') != None:
            book_cover_link.append('https://www.gutenberg.org'+book.find('img').get('src'))
        else:
            book_cover_link.append('notFound')
        if book.find('span',{'class':'title'}) != None:
            book_name.append(book.find('span',{'class':'title'}).string)
        else:
            book_name.append('notFound')
        if book.find('span',{'class':'subtitle'}) != None:
            author.append(book.find('span',{'class':'subtitle'}).string)
        else:
            author.append('notFound')
        if book.find('span',{'class':'extra'}) != None:
            downloads_count.append(book.find('span',{'class':'extra'}).string.split(' ')[0])
        else:
            downloads_count.append('notFound')
    # to get books from subpages I think a recursive function that gets link of page would be good
    if page_soup.find_all('li',{'class':'statusline'})[0].find('a') != None:
        subpages_list=[]
        curr_link=bookshelves_pages_links[i]
        curr_data=requests.get(curr_link)
        curr_soup=bs(curr_data.text,features='html.parser')
        while(curr_soup.find_all('li',{'class':'statusline'})[0].find('a',{'title':'Go to the next page of results.'}) != None):
            curr_link='https://www.gutenberg.org'+curr_soup.find_all('li',{'class':'statusline'})[0].find('a',{'title':'Go to the next page of results.'}).get('href')
            curr_data=requests.get(curr_link)
            curr_soup=bs(curr_data.text,features='html.parser')
            for book in curr_soup.find_all('li',{'class':'booklink'}):
                bookshelves_page.append(bookshelves_pages_titles[i])
                book_link.append('https://www.gutenberg.org'+book.find('a').get('href'))
                if book.find('img') != None:
                    book_cover_link.append('https://www.gutenberg.org'+book.find('img').get('src'))
                else:
                    book_cover_link.append('notFound')
                if book.find('span',{'class':'title'}) != None:
                    book_name.append(book.find('span',{'class':'title'}).string)
                else:
                    book_name.append('notFound')
                if book.find('span',{'class':'subtitle'}) != None:
                    author.append(book.find('span',{'class':'subtitle'}).string)
                else:
                    author.append('notFound')
                if book.find('span',{'class':'extra'}) != None:
                    downloads_count.append(book.find('span',{'class':'extra'}).string.split(' ')[0])
                else:
                    downloads_count.append('notFound')
        
    # to get processing time    
    end_time=time.time()
    time_taken=end_time-start_time
    print('Completed 👏 in',round(time_taken,1),'seconds.')# as the all are taking time in seconds ig

How many bookshelves pages do you want to list? 5
1 )   Bookshelve --> 6 best loved spanish literary classics <--is being processed.......
Completed 👏 in 8.9 seconds.
2 )   Bookshelve --> adventure <--is being processed.......
Completed 👏 in 12.1 seconds.
3 )   Bookshelve --> africa <--is being processed.......
Completed 👏 in 8.6 seconds.
4 )   Bookshelve --> african american writers <--is being processed.......
Completed 👏 in 8.0 seconds.
5 )   Bookshelve --> ainslee's <--is being processed.......
Completed 👏 in 3.5 seconds.


In [205]:
books_df=pd.DataFrame({'Bookshelve Name':bookshelves_page,'Book Name':book_name,'Author':author,'Download Count':downloads_count,'Book Link':book_link,'Book Cover Link':book_cover_link})

In [206]:
books_df

,Bookshelve Name,Book Name,Author,Download Count,Book Link,Book Cover Link
0,6 Best Loved Spanish Literary Classics,Don Quijote (Spanish),Miguel de Cervantes Saavedra,2178,https://www.gutenberg.org/ebooks/2000,https://www.gutenberg.org/cache/epub/2000/pg20...
1,6 Best Loved Spanish Literary Classics,The Life of Lazarillo de Tormes,Anonymous,1188,https://www.gutenberg.org/ebooks/53489,https://www.gutenberg.org/cache/epub/53489/pg5...
2,6 Best Loved Spanish Literary Classics,"The History of Don Quixote, Volume 1, Complete",Miguel de Cervantes Saavedra,1113,https://www.gutenberg.org/ebooks/5921,https://www.gutenberg.org/cache/epub/5921/pg59...
3,6 Best Loved Spanish Literary Classics,"The History of Don Quixote, Volume 2, Complete",Miguel de Cervantes Saavedra,413,https://www.gutenberg.org/ebooks/5946,https://www.gutenberg.org/cache/epub/5946/pg59...
4,6 Best Loved Spanish Literary Classics,Niebla (Nivola) (Spanish),Miguel de Unamuno,329,https://www.gutenberg.org/ebooks/49836,https://www.gutenberg.org/cache/epub/49836/pg4...
...,...,...,...,...,...,...
220,African American Writers,The Anti-Slavery Harp: A Collection of Songs f...,notFound,4,https://www.gutenberg.org/ebooks/10448,https://www.gutenberg.org/cache/epub/10448/pg1...
221,Ainslee's,"Ainslee's, Vol. 15, No. 5, June 1905",Various,26,https://www.gutenberg.org/ebooks/27864,https://www.gutenberg.org/cache/epub/27864/pg2...
222,Ainslee's,"Ainslee's magazine, Volume 16, No. 3, October,...",Various,24,https://www.gutenberg.org/ebooks/47017,https://www.gutenberg.org/cache/epub/47017/pg4...
223,Ainslee's,"Ainslee's, Vol. 15, No. 6, July 1905",Various,12,https://www.gutenberg.org/ebooks/27885,https://www.gutenberg.org/cache/epub/27885/pg2...


In [204]:
## can create clickable links but takes time for these many names
## also can store into a csv or whatever